#### Installing Required Libraries

In [ ]:
! sudo apt install tesseract-ocr ;!pip install python-bidi;!pip install easyocr --no-deps; pip install pytesseract ; pip install pydub ; pip install tesseract ; pip install -U six wheel setuptools ; pip install requests coveralls nose-htmloutput nose coverage ;  

In [ ]:
!sudo apt-get install python-pyaudio python3-pyaudio
!pip install ibm_watson

In [ ]:
 !pip install xlsxwriter; #! pip install wxpython ; 

#### Colab Specific Code (Not for Review).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install easyocr --no-deps

     |████████████████████████████████| 48.9MB 62kB/s 


In [ ]:
!pip install python-bidi

#### Importing Required Libraries.

In [ ]:
#import wx
import re
import xlsxwriter
from io import StringIO
import pytesseract
import sys
import os, os.path
import cv2
import csv
import pandas as pd
import numpy  as np
import re
from bidi.algorithm import get_display
import easyocr
from pydub import AudioSegment
import moviepy.editor as mp
from moviepy.editor import VideoFileClip
import numpy as np
import wave
from IPython.display import Audio
import subprocess
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

#### Generic Function to Make Required Directories.

In [ ]:
def make_dirs(*dirs):
  for directory in dirs:
    os.makedirs(directory, exist_ok = True)

#### Function to extract File Names of all the Videos present inside the folder, based on the accepted extensions.

In [ ]:
def extract_videos(path, extensions):
  #Getting list of All videos 
  files = os.listdir(path)
  vdofiles = []
  vdopaths = []
  for file in files:
    file_arr = file.split('.')
    filename = file_arr[0]
    extension = file_arr[1]
    if extension in extensions:
      vdofiles.append(filename)
      vdopaths.append(os.path.join(path, file))
  return vdofiles, vdopaths

##### Function to Calculate Width and Height of Extracted Text.

In [ ]:
# below function calculate the width and height of text which we have used for headline and tag extraction.
# i am  skipping the height and width calculation part here.
# head line and tagline creation include linguist constarints and custom rules which i will not be sharing in this project.
# Rather i will provide the tag_line and head_line separately, so that you can compare the text acheived with what exactly is required.
def width_and_height_calculator_in_pixel(txt):
    dc   = wx.ScreenDC()
    size = dc.GetTextExtent(txt)
    return size

***Text to speech ***

In [ ]:
# Setup STT Service
# you need to create an account for IBm Watson api key. 
# link :- https://cloud.ibm.com/registration?target=/catalog/services/speech-to-text/&cm_mmc=Earned--Watson%20Core%20-%20Platform--WW_WW-_-intercom&cm_mmca1=000000OF&cm_mmca2=10000409&_ga=2.9387954.643850735.1605631076-2021005727.1604615768
apikey = 'Enter you IBM WAtson Key Here' # You need to put your Api key here
url = 'https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/a31ed939-92c8-44ef-9915-29c35628158f'
# Setup service
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

In [ ]:
#function to get text from the audio file.
def Speech_to_Text(Audio_path):
  with open(Audio_path, 'rb') as f:
    text = stt.recognize(audio=f, content_type='audio/wav', model='en-US_ShortForm_NarrowbandModel',split_transcript_at_phrase_end=True).get_result()
  return(text)


In [ ]:
##Use below lines to understand the output structure of Sppech to text output
#if you want to use any other audio file just replace the below audio path with the path of your file.
#Audio_path='/content/drive/My Drive/Extracted_content/VDO-AD-105/Normalised_Audio/VDO-AD-105_Normalized_Audio.wav'
#text_from_audio=Speech_to_Text(Audio_path)
#len(text_from_audio['results'])
#for i in range(len(text_from_audio['results'])):
  #print(text_from_audio['results'][i]['alternatives'][0]['transcript'])
  #text_from_audio['results'][1]['alternatives'][0]['confidence']

#### Extracting Text from Frames (OCR) And Text from Speech

> Easy_OCR

> IBM_Watson





In [ ]:
def ocr(Content_folder, video_file,worksheet_F,Row_F,Col_F):
  Row_F=0
  Col_F=0
  reader = easyocr.Reader(['en'])
  directory=os.path.join(Content_folder,video_file,"Resized_Gray_frames")
  Frame_txt=os.listdir(directory)
  
  #Path of text folder
  Txt_Frame_path=os.path.join(Content_folder,video_file,"Text_OCR")
  make_dirs(Txt_Frame_path)

  
  #Creating file name with extension
  txt_vdo_xlsx = video_file + ".xlsx"

  #Creating a text(xlsx) file with respective video_name
  workbook = xlsxwriter.Workbook(os.path.join(Txt_Frame_path,txt_vdo_xlsx)) 

                      ##         SPEECH TO TEXT CONVERSION          ##

  #Normalised audio_path for speech to text conversion
  norm_audio_name=video_file+"_Normalized_Audio.wav"
  norm_audio_path=os.path.join(Content_folder,video_file,"Normalised_Audio",norm_audio_name)
  # calling Speect to text conversion function
  text_from_audio=Speech_to_Text(norm_audio_path)
  #text_from_audio
  #len(text_from_audio['results'])

                                     ##      OCR     ##

  # Extracting the text from each frames using easy ocr and writing in excel file
  for frame in Frame_txt:
      #Path of current Resized Gray frame
      frame_path=os.path.join(directory,frame)

      #   "PY-TESSERACT" Code
      # If you  want to use pytesseract use the below lines of code
      #reading the frames for pytesseract
      #img = cv2.imread(frame_path,0)
      # identifies the text in an image
      # It;s good to prefer the adaptive thresholding because of different illumination levels in image.
      #img=cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,93,11) 
      #path of ocr_setup folder
      #text = pytesseract.image_to_string(img)

                                   ##          Easy_OCr           ##
                                   
      text=reader.readtext(frame_path,add_margin=0.15,contrast_ths=0.08,adjust_contrast=1.5,width_ths=0.9,link_threshold=0.8,decoder='beamsearch',paragraph=True,detail = 0)
      #converting the extracted text to file(To read the textfile line by line)
      #f_text = StringIO(text)
      
      #Reading text line by line
      #lines = f_text.readlines()
      #Creating Worksheet to save text of current frame.
      worksheet = workbook.add_worksheet(frame[13:])

      #Defining the headers of each sheet
      row=0
      col=0
      worksheet.write(row, col,"VDO-ID")
      worksheet.write(row, col+1,"Line_Num")
      worksheet.write(row,col+2,"Frame_name")
      worksheet.write(row, col+3,"Line_Text")
      worksheet.write(row,col+4,"Probability_Level")
      #worksheet.write(row, col+5,"Word_Width")
      #worksheet.write(row, col+6,"Word_height")
      
      #Creating Worksheet to save text of current frame.
      Chng_Vid_Count=0

      if len(text)==0:
        #print("No text in ", frame)
        pass
      else:
        # for loop to access each line  
        for line in text:
          line.strip()
          if len(line.strip())<10:
            pass
          else:
            #word_width,word_height= width_and_height_calculator_in_pixel(line)
            Line_num=str(Chng_Vid_Count+1)
            worksheet_F.write(Row_F+1, Col_F,video_file) # VDO_Name
            worksheet_F.write(Row_F+1, Col_F+1, ("line"+Line_num))# line number
            worksheet_F.write(Row_F+1, Col_F+2, frame)  #Frame name
            worksheet_F.write(Row_F+1, Col_F+3, line)   # text in line
            worksheet_F.write(Row_F+1, Col_F+4,"NA" )  #confidence level
            #worksheet_F.write(row+1, Col_F+5,"TBD")   #"Word_ width"
            #worksheet_F.write(row+1, Col_F+6,"TBD")   #Word_height
            Chng_Vid_Count=Chng_Vid_Count+1
            Row_F=Row_F+1
 
        text_frame=reader.readtext(frame_path,add_margin=0.15,contrast_ths=0.08,adjust_contrast=1.5,width_ths=0.9,link_threshold=0.8,decoder='beamsearch')
        #text_ocr_file_Frame wise
        if len(text_frame)==0:
          #print("No text in ", frame)
          pass
        else:
          # for loop to access each line  
          for line in text_frame:
            #word_width,word_height= width_and_height_calculator_in_pixel(line)
            Line_num=str(row+1)
            worksheet.write(row+1, col,video_file) # VDO_Name
            worksheet.write(row+1, col+1, ("line"+Line_num))# line number
            worksheet.write(row+1, col+2, frame)  #Frame name
            worksheet.write(row+1, col+3, line[1].strip())   # text in line
            worksheet.write(row+1, col+4, line[2])  #confidence level
            #worksheet.write(row+1, col+5,"TBD")   #"Word_ width"
            #worksheet.write(row+1, col+6,"TBD")   #Word_height
            row=row+1  
  #here we are using the text_from_audio(Speech_to_text output) and writing the same in Final excel file .
  for entry in range(len(text_from_audio['results'])):
    Audio_text=text_from_audio['results'][entry]['alternatives'][0]['transcript']
    Text_Confidence=text_from_audio['results'][entry]['alternatives'][0]['confidence']
    worksheet_F.write(Row_F+1, Col_F,video_file) # VDO_Name
    worksheet_F.write(Row_F+1, Col_F+1,entry)# Line  Number meaning sentence number
    worksheet_F.write(Row_F+1, Col_F+2, (video_file+"_Audio_Text")) #audio text
    worksheet_F.write(Row_F+1, Col_F+3, Audio_text)  
    worksheet_F.write(Row_F+1, Col_F+4,Text_Confidence )  #confidence level 
    Row_F=Row_F+1    
  workbook.close()
# Note: The final file contains the text output when easy ocr was used with paragraph=TRUE as we will be using the Final file for creating headline and tagline
# We have also saved the frame wise text in respective folder for further text processing.

#### Main Function.

In [ ]:
def main():
  #app = wx.App()
  root_folder ="/content/drive/My Drive/vid"
  Content_folder="/content/drive/My Drive/Extracted_content"
  VdoAD = os.path.join(root_folder, "VDO-AD")
  make_dirs(VdoAD)
  extensions = ['mp4', 'mkv','avi']
  video_files, video_paths = extract_videos(VdoAD, extensions)
  #Creating a workbook_F(xlsx) file 
  workbook_F = xlsxwriter.Workbook("/content/drive/My Drive/Extracted_content/_Final.xlsx") 
  worksheet_F = workbook_F.add_worksheet("First")
  Row_F=0    
  Col_F=0 
  worksheet_F.write(Row_F, Col_F,"VDO-ID")
  worksheet_F.write(Row_F, Col_F+1,"Line_Num")
  worksheet_F.write(Row_F,Col_F+2,"Frame_name")
  worksheet_F.write(Row_F, Col_F+3,"Line_Text")
  worksheet_F.write(Row_F,Col_F+4,"Probability_Level")
  #worksheet_F.write(Row_F, Col_F+5,"Word_Width")
  #worksheet_F.write(Row_F,Col_F+6,"Word_height")
  #Defining the headers of each sheet
  for itr in range(len(video_paths)):
    ocr(Content_folder, video_files[itr],worksheet_F,Row_F,Col_F)
  workbook_F.close()

In [ ]:
def draw_boxes(image, bounds, color='blue', width=2):
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        p0, p1, p2, p3 = bound[0]
        draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
    return image

In [ ]:
main()